In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import pickle

In [4]:
#load the trained model
model = load_model('model.h5')

#load scaler and encoder
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

with open('label_encode_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('one_hot_encoder_geo.pkl','rb') as file:
    one_hot_encoder_geo = pickle.load(file)

In [5]:
#Example input data
input_data = {"CreditScore": 420,
    "Geography": "Germany",
    "Gender": "Female",
    "Age": 58,
    "Tenure": 1,
    "Balance": 145000.50,
    "NumOfProducts": 1,
    "HasCrCard": 0,
    "IsActiveMember": 0,
    "EstimatedSalary": 35000
    }

In [6]:
geo_encoded = one_hot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\TEJASWI\OneDrive\Documents\GEN AI course\Projects\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [7]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,420,Germany,Female,58,1,145000.5,1,0,0,35000


In [8]:
#Encode the categorical variable
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,420,Germany,0,58,1,145000.5,1,0,0,35000


In [11]:
input_df = pd.concat([input_df.drop('Geography', axis = 1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,420,0,58,1,145000.5,1,0,0,35000,0.0,1.0,0.0


In [12]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-2.40397675, -1.09499335,  1.81609745, -1.38944225,  1.10313092,
        -0.91668767, -1.54035103, -1.02583358, -1.13763226, -0.99850112,
         1.72572313, -0.57638802]])

In [13]:
#Predict churn
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 1s 618ms/step


array([[0.9119288]], dtype=float32)

In [14]:
prediction_prob = prediction[0][0]

if prediction_prob>0.5:
    print("Customer is likely to churn")
else:
    print("Customer is unlikely to churn")

Customer is likely to churn
